In [168]:
import pandas as pd
import numpy as np
import unicodedata
import nltk #pip install nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import Levenshtein as lev#pip install python-Levenshtein
def norme(char):
    return unicodedata.normalize('NFD', str(char).lower()).encode('ascii', 'ignore').decode('ascii')

def clean_sentence(sentence):
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [w for w in tokens if not w in stopwords.words('english')]
    return norme(" ".join(filtered_words))

def compute_levenshtein(row):
    data_match_1 = [el for el in data_match if el[0] != row['id']]
    scores = []
    for id, maching in data_match_1:
        distance = lev.distance(row['match_column'], maching)
        scores.append((id, distance))
    return [el[0] for el in sorted(scores, key=lambda x: x[1])[0:2]]

def bests_consine_similarity(x):
    id_score = list(zip(data.id, x))
    return [el[0] for el in sorted(id_score, key=lambda x: x[1], reverse=True)[1:3]]

def combine_similarities(row):
    combined = []
    for x, y in zip(row['similar_cosinus'], row['similar_levenshtein']):
        combined.extend([x, y])
    return list(dict.fromkeys(combined))

data1 = pd.read_csv('/Users/komlan/JungleBike/jb-dev/datasets/products.csv')
data1 = data1.rename(columns={'ean13': 'id', 'product_name':'Name'})
data = data1[['ref_jbin','id','Name']]

data['match_column'] = data.Name.apply(lambda x: clean_sentence(x))

data_match = data[['id','match_column']].values.tolist()

tf_idf = TfidfVectorizer(stop_words='english')
tf_idf_matrix = tf_idf.fit_transform(data.match_column)
cosine_sim = cosine_similarity(tf_idf_matrix, tf_idf_matrix)
data['similar_cosinus'] = list(cosine_sim)
data.similar_cosinus = data.similar_cosinus.apply(lambda x: bests_consine_similarity(x))

data['similar_levenshtein'] = data.apply(lambda row: compute_levenshtein(row), axis=1)
data['combined'] = data.apply(lambda row: combine_similarities(row), axis=1)
data[["similar_name_1", "similar_name_2", "similar_name_3", 'similar_name_4']] = pd.DataFrame(data.combined.tolist(), index = data.index)
data = data.drop(['similar_cosinus','similar_levenshtein','combined'], axis=1)
data

/var/folders/8t/83qwqzx52cs5gmzbyklq6v_w0000gn/T/ipykernel_23696/2374315536.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['match_column'] = data.Name.apply(lambda x: clean_sentence(x))
/var/folders/8t/83qwqzx52cs5gmzbyklq6v_w0000gn/T/ipykernel_23696/2374315536.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['similar_cosinus'] = list(cosine_sim)
/var/folders/8t/83qwqzx52cs5gmzbyklq6v_w0000gn/T/ipykernel_23696/2374315536.py:49: SettingWithCopyWarning: 
A value is trying to be set on a co

,ref_jbin,id,Name,match_column,similar_name_1,similar_name_2,similar_name_3,similar_name_4
0,P295FL000570,4018354006858,Ampoule/Lampe Culot E-10 Graisseur (Feu De Pos...,ampoule lampe culot e 10 graisseur feu de posi...,4018354006841,4.016224e+12,8.590966e+12,NaN
1,P295FL000571,4018354006841,Ampoule/Lampe Culot E-10 Graisseur (Feu De Pos...,ampoule lampe culot e 10 graisseur feu de posi...,4018354006858,4.016224e+12,4.524667e+12,NaN
2,P127AU000581,3700807708891,Antivol Velo Cable A Boucle Auvray Plug Pour F...,antivol velo cable boucle auvray plug pour fer...,3700807708891,3.700808e+12,8.713249e+12,3.700808e+12
3,P127AU000587,3700807700178,Antivol Velo Cable A Cle Auvray,antivol velo cable cle auvray,3700807708136,3.700808e+12,NaN,NaN
4,P201AU000600,3700807700147,Antivol Velo Chaine A Cle Auvray Special Loueu...,antivol velo chaine cle auvray special loueur ...,3700807700147,8.713249e+12,3.700808e+12,8.713249e+12
...,...,...,...,...,...,...,...,...
1292,P86AB007633,4003318868535,Casque Urban-I 3.0,casque urban 3 0,4003318868535,4.003319e+12,4.003319e+12,NaN
1293,P86AB007633,4003318868528,Casque Urban-I 3.0,casque urban 3 0,4003318868535,4.003319e+12,4.003319e+12,NaN
1294,P86AB007633,4003318868603,Casque Urban-I 3.0,casque urban 3 0,4003318868535,4.003319e+12,4.003319e+12,NaN
1295,P86AB007633,4003318868511,Casque Urban-I 3.0,casque urban 3 0,4003318868535,4.003319e+12,4.003319e+12,NaN


In [167]:
data.similar_4.tolist()

[nan,
 nan,
 3700807707894.0,
 nan,
 8713249278049.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 3420587011508.0,
 nan,
 nan,
 nan,
 3420580007218.0,
 nan,
 nan,
 nan,
 8715019177422.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 4524667326571.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 3420580007218.0,
 nan,
 3660429386405.0,
 3660429386511.0,
 nan,
 3660429386405.0,
 3660429386467.0,
 nan,
 3660429386412.0,
 nan,
 3660429386351.0,
 nan,
 nan,
 nan,
 nan,
 4019238107715.0,
 nan,
 nan,
 nan,
 4524667948148.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 710845709418.0,
 710845002908.0,
 nan,
 nan,
 nan,
 nan,
 710845002908.0,
 5604415085796.0,
 4710944248227.0,
 nan,
 nan,
 nan,
 710845002908.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 710845019517.0,
 710845644689.0,
 nan,
 nan,
 nan,
 nan,
 nan

In [282]:
import psycopg2# pip install psycopg2
import paramiko #pip install paramiko
from io import StringIO
import pymysql as mysql #pip install pymysql
from sshtunnel import SSHTunnelForwarder #pip install sshtunnel
ssh_host = 'enovsky.com'
ssh_username = 'enovsky'
host = 'localhost'
username = 'enovsky_psql'
password = 'dakomaje00'
database = 'enovsky_cv_base'
password_base = '6Sqd7uGVJN7fHU39'
port = 5432
server = SSHTunnelForwarder((ssh_host, 22), ssh_username = ssh_username, ssh_password = password, remote_bind_address=(host, port))
server.start()
conn = psycopg2.connect(host=host, database=database, user=username, password=password_base, port=server.local_bind_port)
print('connected')

connected


In [295]:
import pandas as pd
import psycopg2# pip install psycopg2
import unicodedata

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer#pip install scikit-learn
from sklearn.metrics.pairwise import cosine_similarity

import nltk #pip install nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import json


def norme(char):
    return unicodedata.normalize('NFD', str(char).lower()).encode('ascii', 'ignore').decode('ascii')

def get_data():
    query = '''select distinct coun.name as country, ins.name as institution, str.name as structure, dep.name as departement,
                mo.id as id_module, mo.name, mo.full_name from content con
                        inner join module mo on mo.id = con.id_module
                        inner join departement_semester_module dsm on dsm.id_module  = mo.id
                        inner join departement dep on dep.id = dsm.id_departement
                        inner join structure str on str.id = dep.id_structure
                        inner join institution ins on ins.id = str.id_institution
                        inner join country coun on coun.id = ins.id_country'''
    cursor = conn.cursor()
    return pd.read_sql_query(query, con=conn)

def get_user_subscribes():
    id_user = 2
    query = '''with user_module as (
                select con.id_module, max(sub.updated_date)::varchar as last_view from subscription sub
                    inner join content con on con.id = sub.id_content
                   group by sub.id_student, con.id_module having sub.id_student = 2 
                   ),
                user_dep_module as (
                select distinct mo.id as id_module, null as last_view from module mo 
                inner join departement_semester_module dsm on dsm.id_module = mo.id 
                where dsm.id_departement = 36
                )
                select id_module, last_view from user_module
                union
                select id_module, last_view from user_dep_module
                order by last_view desc;'''
    cursor = conn.cursor()
    return pd.read_sql_query(query, con=conn)

def clean_sentence(sentence):
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [w for w in tokens if not w in stopwords.words('french')]
    return norme(" ".join(filtered_words))

def bests_similarity(x):
    id_score = list(zip(data.id_module, x))
    return [el[0] for el in sorted(id_score, key=lambda x: x[1], reverse=True)]


data = get_data()
#data = data.drop_duplicates(subset=['id_module'])

data = data.groupby(['id_module']).agg(lambda rows: ', '.join(set(rows))).reset_index()
data['content'] = data.full_name.apply(lambda x: clean_sentence(x))
data

,id_module,country,institution,structure,departement,name,full_name,content
0,6,TGO,UL,"ENSI, FDS, CIC","GEM, GEC, MRI, GE-CTT, GEL, GC-CTT, MTH, GL, P...",MTH 101,Algèbre linéaire de Base,algebre lineaire base
1,15,TGO,UL,"FDS, CIC","MTH, GL, PHY, MRI, CHM",MTH 104,Calcul Intégral et Applications,calcul integral applications
2,143,TGO,UL,"ENSI, FDS","GE-CTT, GC-CTT, MTH, PHY, CHM, GM-CTT",PHY 106,Mécanique du point matériel,mecanique point materiel
3,940,TGO,UL,FASEG,"MST, OGRH, EIT, APE, CCA, EDE",ECO 310C,Principes de finances publiques,principes finances publiques
4,941,TGO,UL,FASEG,"EIT, APE, EDE",ECO 330C,Economie monétaire,economie monetaire
...,...,...,...,...,...,...,...,...
103,2766,TGO,ISDI,ISDI,"DRV, DRP",DRT2,Procédures pénales,procedures penales
104,2769,TGO,ISDI,ISDI,"DRV, DRP",DRT2,Droit commercial général,droit commercial general
105,2792,TGO,ISDI,ISDI,DRP,DRP3,Droit administratif des biens,droit administratif biens
106,2793,TGO,ISDI,ISDI,DRP,DRP3,Droit communautaire matériel,droit communautaire materiel


In [296]:
user_module = get_user_subscribes()
#
user_module

reco_ids_module

array([  15, 2051, 1237, 1418,  143])

In [323]:
nb_recom = 5
id_departement = 36
user_module = get_user_subscribes()
ids_module = list(set(user_module.id_module.values))
reco_ids_module = user_module[~user_module.last_view.isna()].id_module.values[0:5]

tf_idf = TfidfVectorizer(stop_words='english')
tf_idf_matrix = tf_idf.fit_transform(data.content)
cosine_similarity_matrix = cosine_similarity(tf_idf_matrix, tf_idf_matrix)
data['similars'] = list(cosine_similarity_matrix)

data_reco = data[data.id_module.isin(reco_ids_module)]
data_reco = pd.merge(pd.DataFrame(reco_ids_module, columns=['id_module']), data_reco, on='id_module', how='inner')
data_reco.similars = data_reco.similars.apply(lambda x: bests_similarity(x))
data_reco.similars = data_reco.similars.apply(lambda x: list([el for el in x if el not in ids_module]))

winner_modules = np.transpose(data_reco.similars.tolist()).flatten()
_, indexs = np.unique(winner_modules, return_index=True)
winner_modules = winner_modules[np.sort(indexs)][:20]
selected_columns = ['country','institution','structure','departement', 'id_module', 'name','full_name']
data_winner = pd.merge(pd.DataFrame(winner_modules, columns=['id_module']), data[data.id_module.isin(winner_modules)][selected_columns], on='id_module', how='inner')
data_winner

,id_module,country,institution,structure,departement,name,full_name
0,942,TGO,UL,FASEG,"MST, EIT, APE, CCA, EDE",ECO 304C,Calcul des probabilités
1,2509,TGO,UL,ISICA,"CDO, PAG, JRN",ANG 1269,Anglais : lire et écrire
2,1064,TGO,UL,FDD,DRV,DRV 427,"Régimes matrimoniaux (Cours et TD) (DAFF,DCJD,..."
3,940,TGO,UL,FASEG,"MST, OGRH, EIT, APE, CCA, EDE",ECO 310C,Principes de finances publiques
4,2793,TGO,ISDI,ISDI,DRP,DRP3,Droit communautaire matériel
5,1014,TGO,UL,FDD,"DRV, DRP",FRA 100,Français : Ecouter et parler (UE au Choix)
6,941,TGO,UL,FASEG,"EIT, APE, EDE",ECO 330C,Economie monétaire
7,943,TGO,UL,FASEG,"EIT, APE, EDE",ECO 303C,Equations différentielles
8,944,TGO,UL,FASEG,"MST, OGRH, EIT, APE, CCA, EDE",DRVX 1C,Introduction au droit privé
9,945,TGO,UL,FASEG,"EIT, APE, EDE",ECO 301C,Analyse microéconomique


In [310]:
data_reco.similars.tolist()

[[942,
  940,
  941,
  943,
  944,
  945,
  950,
  951,
  954,
  957,
  1007,
  1009,
  1011,
  1012,
  1013,
  1014,
  1015,
  1031,
  1036,
  1038,
  1039,
  1041,
  1042,
  1051,
  1052,
  1062,
  1064,
  1066,
  1067,
  1068,
  1069,
  1070,
  1071,
  1074,
  1078,
  1084,
  1085,
  1096,
  1126,
  1905,
  1907,
  1908,
  1910,
  1912,
  1913,
  1914,
  1917,
  2133,
  2136,
  2137,
  2139,
  2140,
  2487,
  2488,
  2489,
  2490,
  2492,
  2494,
  2495,
  2496,
  2499,
  2500,
  2504,
  2505,
  2507,
  2508,
  2509,
  2510,
  2511,
  2735,
  2738,
  2740,
  2751,
  2753,
  2762,
  2763,
  2764,
  2765,
  2766,
  2769,
  2793,
  2797],
 [2509,
  1014,
  940,
  941,
  942,
  943,
  944,
  945,
  950,
  951,
  954,
  957,
  1007,
  1009,
  1011,
  1012,
  1013,
  1015,
  1031,
  1036,
  1038,
  1039,
  1041,
  1042,
  1051,
  1052,
  1062,
  1064,
  1066,
  1067,
  1068,
  1069,
  1070,
  1071,
  1074,
  1078,
  1084,
  1085,
  1096,
  1126,
  1905,
  1907,
  1908,
  1910,
  1912,
  1

array([ 942, 2509, 1064,  940, 2793,  940, 1014,  940,  941,  940,  941,
        940,  941,  942,  941,  943,  941,  942,  943,  942,  944,  942,
        943,  944,  943,  945,  943,  944,  945,  944,  950,  944,  945,
        950,  945,  951,  945,  950,  951,  950,  954,  950,  951,  954,
        951,  957,  951,  954,  957,  954, 1007,  954,  957, 1007,  957,
       1009,  957, 1007, 1009, 1007, 1011, 1007, 1009, 1011, 1009, 1012,
       1009, 1011, 1012, 1011, 1013, 1011, 1012, 1013, 1012, 1014, 1012,
       1013, 1014, 1013, 1015, 1013, 1014, 1015, 1014, 1031, 1015, 1015,
       1031, 1015, 1036, 1031, 1031, 1036, 1031, 1038, 1036, 1036, 1038,
       1036, 1039, 1038, 1038, 1039, 1038, 1041, 1039, 1039, 1041, 1039,
       1042, 1041, 1041, 1042, 1041, 1051, 1042, 1042, 1051, 1042, 1052,
       1051, 1051, 1052, 1051, 1062, 1052, 1052, 1062, 1052, 1064, 1062,
       1062, 1064, 1062, 1066, 1064, 1066, 1066, 1064, 1067, 1066, 1067,
       1067, 1066, 1068, 1067, 1068, 1068, 1067, 10

array([ 942, 2509, 1064,  940, 2793, 1014,  941,  943,  944,  945,  950,
        951,  954,  957, 1007, 1009, 1011, 1012, 1013, 1015, 1031, 1036,
       1038, 1039, 1041, 1042, 1051, 1052, 1062, 1066, 1067, 1068, 1069,
       1070, 1071, 1074, 1078, 1084, 1085, 1096, 1126, 1905, 1907, 1908,
       1910, 1912, 1913, 1914, 1917, 2133, 2136, 2137, 2139, 2140, 2487,
       2488, 2489, 2490, 2492, 2494, 2495, 2496, 2499, 2500, 2504, 2505,
       2507, 2508, 2510, 2511, 2735, 2738, 2740, 2751, 2753, 2762, 2763,
       2764, 2765, 2766, 2769, 2797])

In [245]:
selected_columns = ['country','institution','structure','departement','id_departement','id_module', 'name','full_name']
data_similars = []
for i, row in data_reco.iterrows():
    element = json.loads(row.to_json())
    similars = pd.merge(pd.DataFrame(element['similars'], columns=['id_module']), data[data.id_module.isin(element['similars'])][selected_columns], on='id_module', how='inner')
    element['similars_modules'] = json.loads(similars.to_json(orient="records"))
    data_similars.append(element)
data_similars

[{'id_module': 1412,
  'country': 'TGO',
  'institution': 'UL',
  'structure': 'FDS',
  'departement': 'MTH',
  'id_departement': 36,
  'name': 'INF 201',
  'full_name': "Complément d'algorithmique",
  'content': 'complement algorithmique',
  'similars': [2764, 2133, 1917, 1403, 1126, 1039, 940, 1009, 1064, 951],
  'similars_modules': [{'id_module': 2764,
    'country': 'TGO',
    'institution': 'ISDI',
    'structure': 'ISDI',
    'departement': 'DRV',
    'id_departement': 69,
    'name': 'DRT2',
    'full_name': 'Libertés publiques et droits'},
   {'id_module': 2133,
    'country': 'TGO',
    'institution': 'UL',
    'structure': 'FSHS',
    'departement': 'SOC',
    'id_departement': 50,
    'name': 'SOC 115',
    'full_name': 'Sociologie contemporaine'},
   {'id_module': 1917,
    'country': 'TGO',
    'institution': 'UL',
    'structure': 'FSHS',
    'departement': 'HIS',
    'id_departement': 45,
    'name': 'HIS 220',
    'full_name': 'Afrique centrale et orientale du VII è au 

,id_module,country,institution,structure,departement,id_departement,full_name
0,2496,TGO,UL,ISICA,PAG,65,"Relations publiques, relations presses"
1,2489,TGO,UL,ISICA,CDO,63,Histoire des médias
2,1015,TGO,UL,FDD,DRP,29,"Institutions administratives (REDP,ADGL)"
3,944,TGO,UL,FASEG,CCA,23,Introduction au droit privé
4,2500,TGO,UL,ISICA,JRN,64,Etude des textes fondamentaux en SIC
5,2510,TGO,UL,ISICA,JRN,64,Initiation aux tableurs
6,2511,TGO,UL,ISICA,CDO,63,Droit de l'Information et de la communication
7,1914,TGO,UL,FSHS,HIS,45,Géographie physique générale
8,940,TGO,UL,FASEG,CCA,23,Principes de finances publiques
9,1007,TGO,UL,FDD,DRP,29,"Les fondamentaux de la microéconomie (REDP,ADGL)"


In [223]:

#reference = {'splited': 1, 'converted': 2, 'classified': 3, 'cleaned': 4}
ids_module = dict([(el[1], el[0]) for el in list(enumerate(ids_module))])
similars.sort(key=lambda x: ids_module[x])

AttributeError: 'str' object has no attribute 'sort'

In [151]:

def best_consine_similarity(x):
    id_score = list(zip(data.id, x))
    return [el[0] for el in sorted(id_score, key=lambda x: x[1], reverse=True)[0:2]]

data.cosine = data.cosine.apply(lambda x: best_consine_similarity(x))
data

/var/folders/8t/83qwqzx52cs5gmzbyklq6v_w0000gn/T/ipykernel_23696/513484369.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cosine'] = list(cosine_sim)
/var/folders/8t/83qwqzx52cs5gmzbyklq6v_w0000gn/T/ipykernel_23696/513484369.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.cosine = data.cosine.apply(lambda x: best_consine_similarity(x))


,ref_jbin,id,Name,match_column,cosine
0,P295FL000570,4018354006858,Ampoule/Lampe Culot E-10 Graisseur (Feu De Pos...,ampoule lampe culot e 10 graisseur feu de posi...,"[4018354006858, 4018354006841]"
1,P295FL000571,4018354006841,Ampoule/Lampe Culot E-10 Graisseur (Feu De Pos...,ampoule lampe culot e 10 graisseur feu de posi...,"[4018354006841, 4018354006858]"
2,P127AU000581,3700807708891,Antivol Velo Cable A Boucle Auvray Plug Pour F...,antivol velo cable boucle auvray plug pour fer...,"[3700807708891, 3700807708891]"
3,P127AU000587,3700807700178,Antivol Velo Cable A Cle Auvray,antivol velo cable cle auvray,"[3700807700178, 3700807708136]"
4,P201AU000600,3700807700147,Antivol Velo Chaine A Cle Auvray Special Loueu...,antivol velo chaine cle auvray special loueur ...,"[3700807700147, 3700807700147]"
...,...,...,...,...,...
1292,P86AB007633,4003318868535,Casque Urban-I 3.0,casque urban 3 0,"[4003318868597, 4003318868535]"
1293,P86AB007633,4003318868528,Casque Urban-I 3.0,casque urban 3 0,"[4003318868597, 4003318868535]"
1294,P86AB007633,4003318868603,Casque Urban-I 3.0,casque urban 3 0,"[4003318868597, 4003318868535]"
1295,P86AB007633,4003318868511,Casque Urban-I 3.0,casque urban 3 0,"[4003318868597, 4003318868535]"


0       4018354006858
1       4018354006841
2       3700807708891
3       3700807700178
4       3700807700147
            ...      
1292    4003318868535
1293    4003318868528
1294    4003318868603
1295    4003318868511
1296    4003318401701
Name: id, Length: 1297, dtype: int64

In [125]:
set(stopwords.words('english'))

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [117]:
dfm = data.values.tolist()
dfm[3][5] = [1,4]
df = pd.DataFrame(dfm, columns=data.columns)
df

,id,Data ID,Name,match_column,similar_cosinus,similar_levenshtein,combined
0,1,AAA713,Capital Amortization Date,capital amortization date,"[3, 2]","[2, 3]","3,2"
1,2,AAA737,Capital Amortization Rate,capital amortization rate,"[1, 3]","[1, 3]","1,3"
2,3,AAA736,First Date of Capital Amortization,first date capital amortization,"[1, 2]","[1, 2]","1,2"
3,4,AAA738,Amii hsg,amii hsg,"[1, 2]","[1, 4]","1,2"


In [121]:
df['combined'] = df.apply(lambda row: combine_similarities(row), axis=1)
df

,id,Data ID,Name,match_column,similar_cosinus,similar_levenshtein,combined
0,1,AAA713,Capital Amortization Date,capital amortization date,"[3, 2]","[2, 3]","3,2"
1,2,AAA737,Capital Amortization Rate,capital amortization rate,"[1, 3]","[1, 3]","1,3"
2,3,AAA736,First Date of Capital Amortization,first date capital amortization,"[1, 2]","[1, 2]","1,2"
3,4,AAA738,Amii hsg,amii hsg,"[1, 2]","[1, 4]","1,2,4"


array([[1.        , 0.50466766, 1.        , 0.        ],
       [0.50466766, 1.        , 0.50466766, 0.        ],
       [1.        , 0.50466766, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

,id,Data ID,Name,match_column,similar_cosinus,similar_levenshtein,combined,cosine
0,1,AAA713,Capital Amortization Date,capital amortization date,"[3, 2]","[2, 3]","[3, 2]","[1.0000000000000002, 0.504667658050028, 1.0000..."
1,2,AAA737,Capital Amortization Rate,capital amortization rate,"[1, 3]","[1, 3]","[1, 3]","[0.504667658050028, 1.0, 0.504667658050028, 0.0]"
2,3,AAA736,First Date of Capital Amortization,first date capital amortization,"[1, 2]","[1, 2]","[1, 2]","[1.0000000000000002, 0.504667658050028, 1.0000..."
3,4,AAA738,Amii hsg,amii hsg,"[1, 2]","[1, 2]","[1, 2]","[0.0, 0.0, 0.0, 1.0000000000000002]"


In [122]:
df.join(df['combined'].str.split(',', 1, expand=True).rename(columns={0:'combined_1', 1:'combined_2', 2: 'combined_3', 3:'combined_4'}))


/var/folders/8t/83qwqzx52cs5gmzbyklq6v_w0000gn/T/ipykernel_23696/2330061133.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df.join(df['combined'].str.split(',', 1, expand=True).rename(columns={0:'combined_1', 1:'combined_2', 2: 'combined_3', 3:'combined_4'}))


,id,Data ID,Name,match_column,similar_cosinus,similar_levenshtein,combined,combined_1,combined_2
0,1,AAA713,Capital Amortization Date,capital amortization date,"[3, 2]","[2, 3]","3,2",3,2
1,2,AAA737,Capital Amortization Rate,capital amortization rate,"[1, 3]","[1, 3]","1,3",1,3
2,3,AAA736,First Date of Capital Amortization,first date capital amortization,"[1, 2]","[1, 2]","1,2",1,2
3,4,AAA738,Amii hsg,amii hsg,"[1, 2]","[1, 4]","1,2,4",1,"2,4"


In [36]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
text1 = 'Capital Amortization Date'
text2 = 'Capital Amortization Rate'
text3 = 'First Date of Capital Amortization'

# Convert the texts into TF-IDF vectors
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([text1, text2])
# Calculate the cosine similarity between the vectors
similarity = cosine_similarity(vectors)[0][1]
similarity

0.5031026124151314

,ref_jbin,ean13,product_name
0,P295FL000570,4018354006858,Ampoule/Lampe Culot E-10 Graisseur (Feu De Pos...
1,P295FL000571,4018354006841,Ampoule/Lampe Culot E-10 Graisseur (Feu De Pos...
2,P127AU000581,3700807708891,Antivol Velo Cable A Boucle Auvray Plug Pour F...
3,P127AU000587,3700807700178,Antivol Velo Cable A Cle Auvray
4,P201AU000600,3700807700147,Antivol Velo Chaine A Cle Auvray Special Loueu...
...,...,...,...
1292,P86AB007633,4003318868535,Casque Urban-I 3.0
1293,P86AB007633,4003318868528,Casque Urban-I 3.0
1294,P86AB007633,4003318868603,Casque Urban-I 3.0
1295,P86AB007633,4003318868511,Casque Urban-I 3.0


In [140]:
list(data1.columns)

['Unnamed: 0.2',
 'Unnamed: 0',
 'unnamed: 0',
 'ean13',
 'desactiver',
 'new-cat',
 'cat-2',
 'valve-type',
 'output_type',
 'use_part',
 'tire-type',
 'shoe_comp_cleats',
 'motor-position',
 'comp_material',
 'lenses_type',
 'protection_level',
 'sleeve_length',
 'suspender',
 'caliper-type',
 'rubber-type',
 'rubber-standard',
 'brake_type',
 'link-nb',
 'type',
 'comp_brand',
 'comp_model',
 'comp-ean',
 'season',
 'ground-type',
 'rod-type',
 'profil_ext',
 'profil_int',
 'profil_top',
 'nb_holes',
 'offset',
 'stem-type',
 'steerer-type',
 'thread',
 'steerer-diam-inch-bottom',
 'steerer-diam-inch-top',
 'thread-model',
 'comp-bottom-brackets',
 'universal',
 'brake-type.1',
 'rack-width',
 'diameter_stem',
 'diameter_handlebar',
 'diam_seatpost',
 'diameter_frame',
 'speed_nb',
 'compatible_product',
 'product_complement',
 'light_power',
 'teeth_nb_detail',
 'wheel_inch_x',
 'chainwheel-type',
 'cat-1',
 'cat_keys',
 'index',
 'remove_activity',
 'keep_gender',
 'cat_id',
 'cat

In [174]:

def find_depth(dic):
    str_dic = str(dic)
    counter = 0
    for i in str_dic:
        if i == "{":
            counter += 1
    return counter
tree

{1: {}, 2: {1: {}, 7: {}}, 4: {2: {1: {}, 7: {}}}, 7: {}}

In [150]:
max(tree, key=tree.get)

TypeError: '>' not supported between instances of 'dict' and 'dict'

In [94]:



good_tree = tree[max_depth]

#good_tree[max_depth] = good_tree
good_tree

{2: {1: {}, 7: {8: {}}}}

In [206]:
def depth(d):
    if isinstance(d, dict):
        return 1+ (max(map(depth, d.values())) if d else 0)
    return 0

def draw_tree(d, i=0):
    for key, value in d.items():
        if len(value) > 0:
            childs = list(value.keys())
            childs.sort()
            print(' '*(max_depth - i)+'  '.join([str(el) for el in childs]))
        if isinstance(value, dict) and value:
            draw_tree(value, i+1)
            
#table = [(1,2), (2,4), (7,2)]

strArr = ["(1,2)", "(2,4)", "(5,7)", "(7,2)", "(9,5)"]
strArr = [literal_eval(el) for el in strArr]
tree_dic = {node:dict() for link in strArr for node in link }


tree_dic = {node:dict() for link in strArr for node in link }

for child, parent in strArr:
    tree_dic[parent].update({child:tree_dic[child]})

#print(tree_dic)
depths = {}
for key, value in tree_dic.items():
    depths[key] = depth(value)
    
root_key = max(depths, key=depths.get)

max_depth = depth(tree_dic[root_key])

draw_tree({'node':{root_key: tree_dic[root_key]}}, i=0)

     4
    2
   1  7
  5
 9


In [216]:

# Program to calculate moving average
arr =  [3, 1, 3, 5, 10, 6, 4, 3, 1]
window_size = 3
  
i = 0
# Initialize an empty list to store moving averages
moving_averages = []
  
# Loop through the array to consider
# every window of size 3
while i < len(arr) - window_size + 1:
    
    # Store elements from i to i+window_size
    # in list to get the current window
    window = arr[i : i + window_size]
  
    # Calculate the average of current window
    window_average = round(sum(window) / window_size, 2)
      
    # Store the average of current
    # window in moving average list
    moving_averages.append(window_average)
      
    # Shift window to right by one position
    i += 1
print(moving_averages)

[2.33, 3.0, 6.0, 7.0, 6.67, 4.33, 2.67]


In [219]:
def find_sliding(current_array):
    mod = len(current_array) % 2
    value = 0
    if mod != 0:
        value = mod
        return current_array[value]
    else:
        i1 = current_array / 2
        i2 = i1 +1
        return current_array[i1-1] + current_array[i2-1] /2

def find_next_sliding(list_median, array, window_size, current_index):
    current_array = []
    i = current_index
    while i < window_size + current_index:
        current_array.append(array[i])
        i += 1
    try:
        current_array.sort()
        median = find_sliding(current_array)
        list_median.append(median)
        current_index += 1
        find_next_sliding(list_median, array, window_size, current_index)
        return 
    except:
        return

def ArrayChallenge(arr):
    window_size = arr[0]
    arr.pop(0)
    i = 0
    array = arr
    init = array[0]
    median_index = array[0]
    list_median = []
    while median_index < window_size:
        if init == 0:
            list_median.append(0)
        else:
            list_median.append(median_index)
            median_index += 1
            i += 1
    
    if window_size  <= len(array):
        find_next_sliding(list_median, array, window_size, 0)

    # code goes here
    return ','.join(map(str, list_median))

# arr = [3, 1, 3, 5, 10, 6, 4, 3, 1]  
arr = [3, 1, 3, 5, 10, 6, 4, 3, 1]
# arr = [5,2,4,6]
#arr = [3, 0, 0, -2, 0, 2, 0, -2]

ArrayChallenge(arr)

'1,2,3,5,6,6,4,3'

In [123]:
lis[2].key()

AttributeError: 'dict' object has no attribute 'key'

In [5]:
import pandas as pd
raw_data = pd.read_csv('archive/bs140513_032310.csv')
for column in ['customer','age','gender','zipcodeOri','merchant','zipMerchant','category']:
    raw_data[column] = raw_data[column].apply(lambda x: x.replace('\'', ''))

raw_data

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,C1093826151,4,M,28007,M348934600,28007,es_transportation,4.55,0
1,0,C352968107,2,M,28007,M348934600,28007,es_transportation,39.68,0
2,0,C2054744914,4,F,28007,M1823072687,28007,es_transportation,26.89,0
3,0,C1760612790,3,M,28007,M348934600,28007,es_transportation,17.25,0
4,0,C757503768,5,M,28007,M348934600,28007,es_transportation,35.72,0
...,...,...,...,...,...,...,...,...,...,...
594638,179,C1753498738,3,F,28007,M1823072687,28007,es_transportation,20.53,0
594639,179,C650108285,4,F,28007,M1823072687,28007,es_transportation,50.73,0
594640,179,C123623130,2,F,28007,M349281107,28007,es_fashion,22.44,0
594641,179,C1499363341,5,M,28007,M1823072687,28007,es_transportation,14.46,0


In [16]:
s = '''AXA, BNP, Credit Agricole, Thales, BPCE, Safran, EDF, ENGIE, GRTG, Bouygues, SFR, Orange, Societe Generale, RATP, SNCF, SFR, LCL, Saint Gobin, MAZA, PWC, Deloit, Carrefour, Air Liquide, Groupama, Carefour, L’OREAL, Michelin, RENAULT, SANOFI, STELLANTIS, VINCI, PSA, Airbus, Boeing, FNAC-DARTY, NAVAL, ARIANEGROUP'''
print('\n'.join(s.split(',')))

AXA
 BNP
 Credit Agricole
 Thales
 BPCE
 Safran
 EDF
 ENGIE
 GRTG
 Bouygues
 SFR
 Orange
 Societe Generale
 RATP
 SNCF
 SFR
 LCL
 Saint Gobin
 MAZA
 PWC
 Deloit
 Carrefour
 Air Liquide
 Groupama
 Carefour
 L’OREAL
 Michelin
 RENAULT
 SANOFI
 STELLANTIS
 VINCI
 PSA
 Airbus
 Boeing
 FNAC-DARTY
 NAVAL
 ARIANEGROUP


In [222]:
import pandas as pd
matrix = [["1A","Iphone",30,"A"],
         ["1A","Faible",20,"A"],
         ["2B","Dell",15,"C"],
         ["2B","HP",34,"D"]]
data = pd.DataFrame(matrix, columns=["Id","Design","Price","Codeclt"])
data[['Price']].values

array([[30],
       [20],
       [15],
       [34]])

In [231]:
12.6666666666666666/13.45

0.9417596034696406

In [ ]:
> Video speed: pass 10min to 5min	
Video_speed = finale_time/initial_time = 0.5
Audio_speed = initial_time/finale_time = 2
ffmpeg -i input.mp4 -vf "setpts=0.9417596034696406*PTS" -filter:a "atempo=1.061842105263158" output.mp4
    
13.27 ==> 12.00

In [229]:

40*0.5/30

0.6666666666666666